# Sentiment Analysis Using Various Approaches

## Lexicon-based approach 
- Unsupervised learning
- Based on calculating sentiment scores of words in a document from lexicons.
- Each word's sentiment is determined, and the scores are combined to calculate the overall sentiment of the sentence. 
- A lexicon is a dictionary that contains a collection of words that is categorized as positive, negative, and neutral by experts. Their scores can change over time.
- Only those words listed in the lexicon will actually be scored.
- Disadvantages: words that are not in the lexicon will not be scored; some lexicons might be better suited for a specific use; it overlooks negation (lexicons only match words and not phrases, ie "not bad" is scored more negative instead of neutral)

In [28]:
import numpy as np 
import pandas as pd
import json
import time
import re

# Text cleaning
from nltk import sent_tokenize, word_tokenize, regexp_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# NLTK Bing Liu Lexicon 
import nltk
# nltk.download('opinion_lexicon')
from nltk.corpus import opinion_lexicon
from nltk.tokenize import word_tokenize 

# VADER 
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.metrics import accuracy_score

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\MJ\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


## Loading a subset of reviews and meta data

In [2]:
n = 1 
total_rows = 0

def process_chunks(file, chunksize = 1000):

    # Setting as global variables
    global n, total_rows  
    
    chunks = pd.read_json(file, lines=True, chunksize = chunksize)
    dfs = []  
    n_chunks = 0

    for chunk in chunks:
        dfs.append(chunk)
        n_chunks += 1  # Count the number of chunks processed
        print(len(chunk), " rows added")
        n += 1 
        total_rows += len(chunk)
        if n_chunks >= 10:  # Process only the first 5 chunks
            break  
            
    print("Done")
    print("Total rows:", total_rows)
    return pd.concat(dfs, ignore_index=True)

In [3]:
reviews = "../data/Home_and_Kitchen.jsonl"
meta = "../data/meta_Home_and_Kitchen.jsonl"

start = time.process_time()

reviews_subset = process_chunks(reviews)

end = time.process_time()
elapsed_time = end - start
print('Created a subset of the reviews dataset')
print('Execution time:', elapsed_time, 'seconds')

print('--------------')
start = time.process_time()

meta_subset = process_chunks(meta)

end = time.process_time()
elapsed_time = end - start
print('Created a subset of the meta dataset')
print('Execution time:', elapsed_time, 'seconds')

1000  rows added
1000  rows added
1000  rows added
1000  rows added
1000  rows added
1000  rows added
1000  rows added
1000  rows added
1000  rows added
1000  rows added
Done
Total rows: 10000
Created a subset of the reviews dataset
Execution time: 0.359375 seconds
--------------
1000  rows added
1000  rows added
1000  rows added
1000  rows added
1000  rows added
1000  rows added
1000  rows added
1000  rows added
1000  rows added
1000  rows added
Done
Total rows: 20000
Created a subset of the meta dataset
Execution time: 1.53125 seconds


In [4]:
reviews_subset.head(3)

,rating,title,text,images,asin,parent_asin,user_id,timestamp,helpful_vote,verified_purchase
0,1,Received Used & scratched item! Purchased new!,Livid. Once again received an obviously used ...,[],B007WQ9YNO,B09XWYG6X1,AFKZENTNBQ7A7V7UXW5JJI6UGRYQ,2023-02-26 01:03:29.298,1,True
1,5,Excellent for moving & storage & floods!,I purchased these for multiple reasons. The ma...,[],B09H2VJW6K,B0BXDLF8TW,AFKZENTNBQ7A7V7UXW5JJI6UGRYQ,2022-12-26 08:30:10.846,0,True
2,2,Lid very loose- needs a gasket imo. Small base.,[[VIDEOID:c87e962bc893a948856b0f1b285ce6cc]] I...,[{'small_image_url': 'https://m.media-amazon.c...,B07RL297VR,B09G2PW8ZG,AFKZENTNBQ7A7V7UXW5JJI6UGRYQ,2022-05-25 02:54:56.788,0,True


In [5]:
reviews_subset.columns

Index(['rating', 'title', 'text', 'images', 'asin', 'parent_asin', 'user_id',
       'timestamp', 'helpful_vote', 'verified_purchase'],
      dtype='object')

## Text Cleaning - spaCy

In [6]:
import spacy 
nlp = spacy.load('en_core_web_sm')
stop_words = spacy.lang.en.stop_words.STOP_WORDS
print('Original stopwords count:', len(stop_words))

Original stopwords count: 326


In [7]:
# Updating my stopwords list 
stop_words = spacy.lang.en.stop_words.STOP_WORDS

# Include/ exclude certain words
include_stopwords = {'would', 'I'}
exclude_stopwords = {'i', 'well', 'off', 'very', 'not', 'few', 'much'}

stop_words |= include_stopwords
stop_words -= exclude_stopwords

# Remove adjectives from my stopwords using spaCy
exclude_adjectives = {word for word in stop_words if nlp(word)[0].pos_ == "ADJ"}
print(exclude_adjectives)

stop_words -= exclude_adjectives
print('Stopwords count:', len(stop_words))

{'serious', 'own', 'same', 'third', 'last', 'whole', 'whereafter', 'many', 'empty', 'least', 'several', 'latter', 'former', 'due', 'other', 'top', 'various', 'full', 'such'}
Stopwords count: 301


In [8]:
def clean_data(doc):
    doc = doc.lower()
    doc = nlp(doc)
    # Lemmatize words 
    lemmas = [token.lemma_ for token in doc]
    # Removing non-alphabetic characters and stopwords
    tokens = [lemma for lemma in lemmas if lemma.isalpha() and lemma not in stop_words]
    cleaned_doc = " ".join(tokens)
    
    return cleaned_doc

In [9]:
stop_words

{"'d",
 "'ll",
 "'m",
 "'re",
 "'s",
 "'ve",
 'I',
 'a',
 'about',
 'above',
 'across',
 'after',
 'afterwards',
 'again',
 'against',
 'all',
 'almost',
 'alone',
 'along',
 'already',
 'also',
 'although',
 'always',
 'am',
 'among',
 'amongst',
 'amount',
 'an',
 'and',
 'another',
 'any',
 'anyhow',
 'anyone',
 'anything',
 'anyway',
 'anywhere',
 'are',
 'around',
 'as',
 'at',
 'back',
 'be',
 'became',
 'because',
 'become',
 'becomes',
 'becoming',
 'been',
 'before',
 'beforehand',
 'behind',
 'being',
 'below',
 'beside',
 'besides',
 'between',
 'beyond',
 'both',
 'bottom',
 'but',
 'by',
 'ca',
 'call',
 'can',
 'cannot',
 'could',
 'did',
 'do',
 'does',
 'doing',
 'done',
 'down',
 'during',
 'each',
 'eight',
 'either',
 'eleven',
 'else',
 'elsewhere',
 'enough',
 'even',
 'ever',
 'every',
 'everyone',
 'everything',
 'everywhere',
 'except',
 'fifteen',
 'fifty',
 'first',
 'five',
 'for',
 'formerly',
 'forty',
 'four',
 'from',
 'front',
 'further',
 'get',
 'give'

In [10]:
lemmatizer = WordNetLemmatizer()
stopwords = set(stopwords.words('english'))

# Update stopwords
include_stopwords = {'would', 'I', 'not'}
exclude_stopwords = {'i'}

stopwords |= include_stopwords
stopwords -= exclude_stopwords

def preprocess_text(text):
    text = text.lower()  # Lowercase text
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # Remove punctuation and numbers
    tokens = text.split()  # Create tokens 
    clean_tokens = [lemmatizer.lemmatize(t) for t in tokens if t not in stopwords]  # Lemmatize and remove stop words
    clean_text = " ".join(clean_tokens)  # Join clean tokens
    clean_text = " ".join(clean_text.split())  # Remove extra spaces, tabs, and new lines
    
    return clean_text

In [11]:
stopwords

{'I',
 'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'only',
 'or',
 'other',
 'our',
 'ours',
 'ourselves',
 'out',
 'over',
 'own',
 'r

In [12]:
cleaned_text = reviews_subset.copy()
cleaned_text['spacy_text'] = cleaned_text['text'].apply(clean_data)
cleaned_text['nltk_text'] = cleaned_text['text'].apply(preprocess_text)

In [13]:
cleaned_text.head(3)

,rating,title,text,images,asin,parent_asin,user_id,timestamp,helpful_vote,verified_purchase,spacy_text,nltk_text
0,1,Received Used & scratched item! Purchased new!,Livid. Once again received an obviously used ...,[],B007WQ9YNO,B09XWYG6X1,AFKZENTNBQ7A7V7UXW5JJI6UGRYQ,2023-02-26 01:03:29.298,1,True,livid receive obviously use item food scratch ...,livid received obviously used item food scratc...
1,5,Excellent for moving & storage & floods!,I purchased these for multiple reasons. The ma...,[],B09H2VJW6K,B0BXDLF8TW,AFKZENTNBQ7A7V7UXW5JJI6UGRYQ,2022-12-26 08:30:10.846,0,True,purchase multiple reason main reason bc apt fl...,i purchased multiple reason main reason i movi...
2,2,Lid very loose- needs a gasket imo. Small base.,[[VIDEOID:c87e962bc893a948856b0f1b285ce6cc]] I...,[{'small_image_url': 'https://m.media-amazon.c...,B07RL297VR,B09G2PW8ZG,AFKZENTNBQ7A7V7UXW5JJI6UGRYQ,2022-05-25 02:54:56.788,0,True,videoid want love bc previously buy matching t...,videoidcebcabfbcecc i wanted love bc i previou...


## Bing Liu Lexicon

The Bing Liu lexicon has a total of 6, 786 words with 2,005 classified as positive and 4,781 as negative. CLassification is binary (positive or negative).

In [14]:
print('Total number of words in opinion lexicon', len(opinion_lexicon.words()))
print('Examples of positive words:', opinion_lexicon.positive()[:10])
print('Examples of negative words:', opinion_lexicon.negative()[:10])

Total number of words in opinion lexicon 6789
Examples of positive words: ['a+', 'abound', 'abounds', 'abundance', 'abundant', 'accessable', 'accessible', 'acclaim', 'acclaimed', 'acclamation']
Examples of negative words: ['2-faced', '2-faces', 'abnormal', 'abolish', 'abominable', 'abominably', 'abominate', 'abomination', 'abort', 'aborted']


In [15]:
pos_score = 1
neg_score = -1
word_dict = {}

# Adding the positive words to the dictionary
for word in opinion_lexicon.positive():
    word_dict[word] = pos_score 

# Adding the negative words to the dictionary 
for word in opinion_lexicon.negative():
    word_dict[word] = neg_score 

def bing_liu_score(text):
    sentiment_score = 0 
    bag_of_words = word_tokenize(text.lower())

    # Check if bag_of_words is empty
    if bag_of_words: 
        for word in bag_of_words: 
            if word in word_dict: 
                sentiment_score += word_dict[word]
        return sentiment_score / len(bag_of_words)
    else: 
        return 0

In [16]:
cleaned_text['Bing_Liu_score'] = cleaned_text['text'].apply(bing_liu_score)
cleaned_text['Bing_Liu_spaCy'] = cleaned_text['spacy_text'].apply(bing_liu_score)
cleaned_text['Bing_Liu_nltk'] = cleaned_text['nltk_text'].apply(bing_liu_score)
cleaned_text.head(5)

,rating,title,text,images,asin,parent_asin,user_id,timestamp,helpful_vote,verified_purchase,spacy_text,nltk_text,Bing_Liu_score,Bing_Liu_spaCy,Bing_Liu_nltk
0,1,Received Used & scratched item! Purchased new!,Livid. Once again received an obviously used ...,[],B007WQ9YNO,B09XWYG6X1,AFKZENTNBQ7A7V7UXW5JJI6UGRYQ,2023-02-26 01:03:29.298,1,True,livid receive obviously use item food scratch ...,livid received obviously used item food scratc...,-0.085714,-0.187500,-0.187500
1,5,Excellent for moving & storage & floods!,I purchased these for multiple reasons. The ma...,[],B09H2VJW6K,B0BXDLF8TW,AFKZENTNBQ7A7V7UXW5JJI6UGRYQ,2022-12-26 08:30:10.846,0,True,purchase multiple reason main reason bc apt fl...,i purchased multiple reason main reason i movi...,-0.006568,-0.021459,-0.019868
2,2,Lid very loose- needs a gasket imo. Small base.,[[VIDEOID:c87e962bc893a948856b0f1b285ce6cc]] I...,[{'small_image_url': 'https://m.media-amazon.c...,B07RL297VR,B09G2PW8ZG,AFKZENTNBQ7A7V7UXW5JJI6UGRYQ,2022-05-25 02:54:56.788,0,True,videoid want love bc previously buy matching t...,videoidcebcabfbcecc i wanted love bc i previou...,-0.008287,-0.029851,-0.016949
3,5,Best purchase ever!,If you live at a higher elevation like me (5k ...,[{'small_image_url': 'https://m.media-amazon.c...,B09CQF4SWV,B08CSZDXZY,AFKZENTNBQ7A7V7UXW5JJI6UGRYQ,2022-05-06 16:38:16.178,0,True,live high elevation like colorado know buzzer ...,live higher elevation like k colorado know buz...,0.006316,-0.029586,0.009615
4,5,Excellent for yarn!,I use these to store yarn. They easily hold 12...,[{'small_image_url': 'https://images-na.ssl-im...,B003U6A3EY,B0C6V27S6N,AFKZENTNBQ7A7V7UXW5JJI6UGRYQ,2020-05-20 00:28:45.940,1,True,use store yarn easily hold ounce bernat pipsqu...,i use store yarn easily hold ounce bernat pips...,0.035294,0.058824,0.075000


In [17]:
cleaned_text[['rating', 'text', 'Bing_Liu_score', 'Bing_Liu_spaCy', 'Bing_Liu_nltk']].sample(5)

,rating,text,Bing_Liu_score,Bing_Liu_spaCy,Bing_Liu_nltk
9682,5,"Perfect holder to dry your bottles, works great.",0.300000,0.500000,0.500000
8967,5,"Large. Feels rock solid, cooks well and has a ...",0.200000,0.333333,0.333333
1296,5,Love it. I have Myasthenia Gravis and it can b...,0.038462,0.111111,0.090909
6572,5,This 4 tier cupcake stand is easy to assemble....,0.050000,0.103448,0.105263
5697,4,So ultimately i think this product is worth it...,0.018519,0.068493,0.040404


In [18]:
# Look for scores of zeroes in Bing_Liu_spaCy and Bing_Liu_nltk columns 
cleaned_text[cleaned_text['Bing_Liu_spaCy'] == 0]

,rating,title,text,images,asin,parent_asin,user_id,timestamp,helpful_vote,verified_purchase,spacy_text,nltk_text,Bing_Liu_score,Bing_Liu_spaCy,Bing_Liu_nltk
15,5,Perfect mattress,"OK, we bought this mattress for our guest room...",[],B0777K9RGX,B0BPBLYF85,AGGZ357AO26RQZVRLGU4D4N52DZQ,2020-01-06 02:16:33.373,0,True,ok buy mattress guest room wish mattress serio...,ok bought mattress guest room im wishing mattr...,0.000000,0.0,0.000000
19,5,Totally. Obsessed. With dehydrating food. Than...,This is really a great dehydrator - and so eas...,[],B000FFVJ3C,B0B69XTPVL,AGGZ357AO26RQZVRLGU4D4N52DZQ,2012-12-05 03:04:18.000,1,True,great dehydrator easy use homemade trail mix u...,really great dehydrator easy use i make homema...,0.000000,0.0,-0.011494
25,4,Nice sparkly touch,Nice quality. A little pricey for the amount b...,[],B01I3Q6BHS,B01I3Q6BHS,AGKASBHYZPGTEPO6LWZPVJWB2BVA,2017-01-30 15:35:05.000,0,True,nice quality little pricey reusable desire,nice quality little pricey amount also reusabl...,0.000000,0.0,0.000000
34,5,Smell My Pumpkin,Smells much better than the Smell My Nuts cand...,[],B004FO2A78,B01KU4IFXO,AGCI7FAH4GL5FI65HYLKWTMFZ2CQ,2014-11-18 06:15:48.000,0,True,smell much well smell nuts candle buy father l...,smell much better smell nut candle i bought fa...,0.000000,0.0,0.000000
35,5,yay,happily loved by our 1 year old. she makes fu...,[],B00563XRYM,B00563XRYM,AGCI7FAH4GL5FI65HYLKWTMFZ2CQ,2013-11-20 01:54:13.000,0,True,happily love year old funny noise think funny ...,happily loved year old make funny noise think ...,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9972,5,Sturdy and decorative.,I love these so much that I've purchased 3 set...,[],B08GTTM7G7,B08GTTM7G7,AE57MRF2R2ALCC6H5WQLFKT7KSSA,2021-12-02 23:08:30.693,0,True,love much purchase set other gift live humid f...,i love much ive purchased set others gift i li...,0.000000,0.0,0.000000
9976,5,Does the job!,Just what I was looking for.,[],B0765BTD13,B0BWJ277SP,AE57MRF2R2ALCC6H5WQLFKT7KSSA,2021-03-05 15:23:36.385,0,True,look,i looking,0.000000,0.0,0.000000
9979,1,IT INTERFERS WITH THE STEAMER BUTTON,It is lightweight and deep enough to steam ent...,[],B079CLQJ5Z,B079CLQJ5Z,AE57MRF2R2ALCC6H5WQLFKT7KSSA,2019-10-05 16:57:37.457,2,True,lightweight deep steam entire head broccoli ca...,lightweight deep enough steam entire head broc...,0.018182,0.0,0.034483
9987,1,Not comfortable,"Uncomfortable, not cool, and disappointed. Eas...",[],B07MY34QH7,B0BLXVK6QX,AFPHKIJFGIU4G4POXRFCEF5RJJHA,2020-12-05 16:13:34.579,1,True,uncomfortable not cool disappoint easy return ...,uncomfortable cool disappointed easy return po...,0.000000,0.0,0.000000


In [19]:
i = 3
print(f'Title: {cleaned_text.loc[i,"title"]}\n')

print(f'Text: {cleaned_text.loc[i,"text"]}\n')

print(f'Text: {cleaned_text.loc[i,"spacy_text"]}\n')

print(f'Text: {cleaned_text.loc[i,"nltk_text"]}')

Title: Best purchase ever!

Text: If you live at a higher elevation like me (5k Colorado) just know that after the buzzer beeps you might wanna leave the hard boiled eggs inside the steamed cooker for another minute or two if you want the egg cooked all the way through. I also add a bit more water than the fill line says, but again that’s bc of my elevation & it evaporates quickly.  It’s not any quicker than I used to make them, but it’s a lot less water. I’m disabled & have a TBI rn so the auto-cutoff was a no-brained for me. I bought the blue bc it was cheapest.  My service 🐶  🐶 love hard boiled eggs too so we are enjoying this new addition to our kitchen.  My only regret is that I did not buy it sooner & that Amazon never has the color I want for the cheapest price, but I can live with the bb blue.  We have used it 3x & so far it’s working okay. Oh- don’t place it under upper cabinets as it has a vent hole & it will vent & it will make the cabinets sweat.  Give it some room to breat

In [20]:
# Calculate mean sentiment score for each rating category
mean_scores = cleaned_text.groupby('rating').agg({
    'Bing_Liu_score':'mean',
    'Bing_Liu_spaCy': 'mean',
    'Bing_Liu_nltk': 'mean'
}).reset_index()

print(mean_scores)

   rating  Bing_Liu_score  Bing_Liu_spaCy  Bing_Liu_nltk
0       1       -0.031370       -0.063742      -0.051328
1       2        0.003011        0.004356       0.009655
2       3        0.031123        0.062282       0.062611
3       4        0.066978        0.143764       0.132249
4       5        0.129455        0.249447       0.227450


In [21]:
# Calculate max sentiment score for each rating category
max_scores = cleaned_text.groupby('rating').agg({
    'Bing_Liu_score':'max',
    'Bing_Liu_spaCy': 'max',
    'Bing_Liu_nltk': 'max'
}).reset_index()

print(max_scores)

   rating  Bing_Liu_score  Bing_Liu_spaCy  Bing_Liu_nltk
0       1        0.333333             0.5            1.0
1       2        0.250000             0.5            1.0
2       3        1.000000             1.0            1.0
3       4        1.000000             1.0            1.0
4       5        1.000000             1.0            1.0


In [22]:
# Calculate min sentiment score for each rating category
min_scores = cleaned_text.groupby('rating').agg({
    'Bing_Liu_score':'min',
    'Bing_Liu_spaCy': 'min',
    'Bing_Liu_nltk': 'min'
}).reset_index()

print(min_scores)

   rating  Bing_Liu_score  Bing_Liu_spaCy  Bing_Liu_nltk
0       1       -1.000000       -1.000000      -1.000000
1       2       -0.666667       -1.000000      -0.666667
2       3       -0.500000       -0.666667      -0.500000
3       4       -1.000000       -1.000000      -1.000000
4       5       -0.500000       -1.000000      -1.000000


## VADER Lexicon
Rule-based lexicon. 
9,000 features with scales of [-4] Extremely Negative to [4] Extremely Positive with [0] for Neutral or Neither. 

In [23]:
model = SentimentIntensityAnalyzer()

In [38]:
def vader_score(text):
    score = model.polarity_scores(text)
    compound_score = score['compound']
    return compound_score

In [39]:
cleaned_text['Vader_score'] = cleaned_text['text'].apply(vader_score)
cleaned_text['Vader_spaCy'] = cleaned_text['spacy_text'].apply(vader_score)
cleaned_text['Vader_nltk'] = cleaned_text['nltk_text'].apply(vader_score)
cleaned_text.head(5)

,rating,title,text,images,asin,parent_asin,user_id,timestamp,helpful_vote,verified_purchase,spacy_text,nltk_text,Bing_Liu_score,Bing_Liu_spaCy,Bing_Liu_nltk,Vader_score,Vader_spaCy,Vader_nltk
0,1,Received Used & scratched item! Purchased new!,Livid. Once again received an obviously used ...,[],B007WQ9YNO,B09XWYG6X1,AFKZENTNBQ7A7V7UXW5JJI6UGRYQ,2023-02-26 01:03:29.298,1,True,livid receive obviously use item food scratch ...,livid received obviously used item food scratc...,-0.085714,-0.187500,-0.187500,-0.8168,-0.8126,-0.7845
1,5,Excellent for moving & storage & floods!,I purchased these for multiple reasons. The ma...,[],B09H2VJW6K,B0BXDLF8TW,AFKZENTNBQ7A7V7UXW5JJI6UGRYQ,2022-12-26 08:30:10.846,0,True,purchase multiple reason main reason bc apt fl...,i purchased multiple reason main reason i movi...,-0.006568,-0.021459,-0.019868,0.9942,0.9672,0.9901
2,2,Lid very loose- needs a gasket imo. Small base.,[[VIDEOID:c87e962bc893a948856b0f1b285ce6cc]] I...,[{'small_image_url': 'https://m.media-amazon.c...,B07RL297VR,B09G2PW8ZG,AFKZENTNBQ7A7V7UXW5JJI6UGRYQ,2022-05-25 02:54:56.788,0,True,videoid want love bc previously buy matching t...,videoidcebcabfbcecc i wanted love bc i previou...,-0.008287,-0.029851,-0.016949,0.9724,0.9292,0.9708
3,5,Best purchase ever!,If you live at a higher elevation like me (5k ...,[{'small_image_url': 'https://m.media-amazon.c...,B09CQF4SWV,B08CSZDXZY,AFKZENTNBQ7A7V7UXW5JJI6UGRYQ,2022-05-06 16:38:16.178,0,True,live high elevation like colorado know buzzer ...,live higher elevation like k colorado know buz...,0.006316,-0.029586,0.009615,0.9967,0.9823,0.9867
4,5,Excellent for yarn!,I use these to store yarn. They easily hold 12...,[{'small_image_url': 'https://images-na.ssl-im...,B003U6A3EY,B0C6V27S6N,AFKZENTNBQ7A7V7UXW5JJI6UGRYQ,2020-05-20 00:28:45.940,1,True,use store yarn easily hold ounce bernat pipsqu...,i use store yarn easily hold ounce bernat pips...,0.035294,0.058824,0.075000,0.9590,0.6085,0.6085
